In [1]:
import matplotlib.pyplot as plt
from glob import glob
from skimage.io import imread
from skimage.color import rgb2gray
from sklearn.model_selection import train_test_split
import keras
import numpy as np

from keras.models import Sequential
from keras.layers import (  Dense,
                            Flatten,
                            LeakyReLU
                         )
from keras.applications import  VGG19 
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from skimage.transform import resize
from tqdm import tqdm
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
def hist(History):
    fig, ax = plt.subplots(1, 2, figsize=(20, 5))

    ax[0].plot(History.history['loss'])
    ax[0].plot(History.history['val_loss'])
    ax[0].legend(['Training loss', 'Validation Loss'],fontsize=18)
    ax[0].set_xlabel('Epochs ',fontsize=16)
    ax[0].set_ylabel('Loss',fontsize=16)
    ax[0].set_title('Training loss x Validation Loss',fontsize=16)


    ax[1].plot(History.history['acc'])
    ax[1].plot(History.history['val_acc'])
    ax[1].legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
    ax[1].set_xlabel('Epochs ',fontsize=16)
    ax[1].set_ylabel('Accuracy',fontsize=16)
    ax[1].set_title('Training Accuracy x Validation Accuracy',fontsize=16)

In [3]:
def plot_any(arr, title = ''):
    plt.figure(figsize = (15, 25))
    for i in range(len(arr)):
        plt.subplot(1,len(arr),i + 1)
        plt.title(title)
        plt.imshow(arr[i], cmap = 'gray');

In [4]:
path = '../input/skin-cancer-malignant-vs-benign/data/'
train_path = glob(path + 'train/*')
test_path = glob(path + 'test/*')

In [5]:
train_imgs = []
train_labels = []
test_imgs  = []
test_labels = []

x, y, z = 224, 224, 3

#train
for klass, folder in enumerate(tqdm(train_path)):
    for img in glob(folder + '/*'):
        
        img_resize = imread(img)
        img_resize = resize(img_resize, (x, y, z))

        train_imgs.append(img_resize)
        train_labels.append(klass)
        
#test
for klass, folder in enumerate(tqdm(test_path)):
    for img in glob(folder + '/*'):
        
        img_resize = imread(img)
        img_resize = resize(img_resize, (x, y, z))

        test_imgs.append(img_resize)
        test_labels.append(klass)

100%|██████████| 2/2 [00:12<00:00,  6.02s/it]


In [6]:
print('Treino: {} \nTeste: {}'.format(len(train_imgs), len(test_imgs)))

Treino: 2637 
Teste: 660


In [7]:
train_imgs = np.asarray(train_imgs)
test_imgs = np.asarray(test_imgs)

train_labels =  np.asarray(train_labels)
test_labels =  np.asarray(test_labels)

In [8]:
NUM_CLASSES = 1
EPOCHS = 15
BATCH_SIZE = 64
inputShape = (x, y, z)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    train_imgs, 
    train_labels,
    test_size = 0.3, 
)

In [10]:
train_datagen = ImageDataGenerator( rescale = 1./255,
                                    rotation_range=90,
                                    width_shift_range=0.15,
                                    height_shift_range=0.15,
                                    horizontal_flip = True,
                                    vertical_flip = True,
                                    zoom_range=(0.9,1),
                                    fill_mode= 'nearest',
                                    brightness_range=(0.8, 1.2),
                                  )

train_generator = train_datagen.flow(X_train, y_train, batch_size = BATCH_SIZE)
val_generator = train_datagen.flow(X_test, y_test, batch_size = BATCH_SIZE, shuffle = True)

In [11]:
model = Sequential()
model.add(VGG19(include_top=False, weights='imagenet', input_shape= inputShape))
model.add(Flatten())
model.add(Dense(32))
model.add(LeakyReLU(0.001))
model.add(Dense(16))
model.add(LeakyReLU(0.001))
model.add(Dense(1, activation='sigmoid'))
model.layers[0].trainable = False

model.compile(loss=keras.losses.binary_crossentropy, optimizer='adam', metrics=['acc'])

History = model.fit_generator(train_generator,
    steps_per_epoch = len(train_imgs) // BATCH_SIZE,
    epochs = EPOCHS, 
    validation_data = val_generator,
    validation_steps = len(test_imgs) // BATCH_SIZE,
)

80142336/80134624 [==============================] - 2s 0us/step
Epoch 1/15
41/41 [==============================] - 44s 1s/step - loss: 0.6009 - acc: 0.7025 - val_loss: 0.4649 - val_acc: 0.8094
Epoch 2/15
41/41 [==============================] - 42s 1s/step - loss: 0.4620 - acc: 0.7773 - val_loss: 0.4254 - val_acc: 0.7883
Epoch 3/15
41/41 [==============================] - 39s 957ms/step - loss: 0.4557 - acc: 0.7811 - val_loss: 0.3551 - val_acc: 0.8050
Epoch 4/15
41/41 [==============================] - 38s 921ms/step - loss: 0.4166 - acc: 0.8048 - val_loss: 0.3836 - val_acc: 0.7983
Epoch 5/15
41/41 [==============================] - 39s 960ms/step - loss: 0.4267 - acc: 0.7880 - val_loss: 0.2891 - val_acc: 0.8594
Epoch 6/15
41/41 [==============================] - 44s 1s/step - loss: 0.3936 - acc: 0.8082 - val_loss: 0.3222 - val_acc: 0.8133
Epoch 7/15
41/41 [==============================] - 36s 867ms/step - loss: 0.3983 - acc: 0.8125 - val_loss: 0.3885 - val_acc: 0.8583
Epoch 8/15
41

In [12]:
loss, accu = model.evaluate(test_imgs, test_labels)
print("%s: %.2f%%" % ('Accuracy...', accu))
print("%s: %.2f" % ('loss.......', loss))

660/660 [==============================] - 3s 5ms/step
Accuracy...: 0.84%
loss.......: 0.34
